# Workforce planning

Job recommendation can be extended across all members of an organisation.

The skills of each individual is compared with the skills of each job profile and an optimal set of suggestions returned.

This notebook will demonstrate this functionality using the users and profiles created in the setup notebook. If you have not already gone through the setup notebook please do so before running the code below.

In [13]:
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000"

In [14]:
# Get organisation and graph details

r = requests.get(base_url + "/api/v1/graphs")
print("Status code", r.status_code)

print("Graph response", r.json())

graph_tag = "test"
assert graph_tag in [i['graph_tag'] for i in r.json()], "Graph not found, run setup notebook"

r = requests.get(base_url + "/api/v1/organisations")
print("Status code", r.status_code)

print("Organisation response", r.json())

# Get organisation ID for the test organisation created during setup
assert 'Test organisation' in [i['name'] for i in r.json()], "Organisation not found, run setup notebook"
organisation_id = [i for i in r.json() if i["name"] == "Test organisation"][0]["id"]

Status code 200
Graph response [{'graph_tag': 'test', 'id': 1, 'name': 'Test graph'}]
Status code 200
Organisation response [{'id': 1, 'name': 'Test organisation'}]


In [16]:
# Get users

r = requests.get(base_url + "/api/v1/organisations/{0}/users".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())
all_users = r.json()
all_users = {i["id"]:i["name"] for i in all_users}

Status code 200
Response [{'id': 1, 'name': 'Adam'}, {'id': 2, 'name': 'Beth'}, {'id': 3, 'name': 'Chris'}, {'id': 4, 'name': 'Dani'}]


In [17]:
# Get profiles

r = requests.get(base_url + "/api/v1/organisations/{0}/profiles".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())
all_profiles = {i["id"]:i["name"] for i in r.json()}

Status code 200
Response [{'id': 1, 'name': 'Applications Engineer'}, {'id': 2, 'name': 'Data Scientist'}, {'id': 3, 'name': 'Computer and Information Systems Managers'}, {'id': 4, 'name': 'Web Developers'}, {'id': 5, 'name': 'Software Engineer'}, {'id': 6, 'name': 'Software Engineer ML'}, {'id': 7, 'name': 'Systems Engineer'}, {'id': 8, 'name': 'DevOps Engineer'}, {'id': 9, 'name': 'Machine Learning Researcher'}, {'id': 10, 'name': 'Business Intelligence Analyst'}, {'id': 11, 'name': 'Software Developers, Applications'}, {'id': 12, 'name': 'Cloud architect'}, {'id': 13, 'name': 'Sales Engineer'}, {'id': 14, 'name': 'Database Administrators'}, {'id': 15, 'name': 'Security Engineer'}, {'id': 16, 'name': 'Computer Hardware Engineers'}, {'id': 17, 'name': 'Data Engineer'}, {'id': 18, 'name': 'Product Manager'}, {'id': 19, 'name': 'UX Designer'}, {'id': 20, 'name': 'Solutions Architect'}, {'id': 21, 'name': 'Mobile Developer'}, {'id': 22, 'name': 'Machine Learning Engineer'}, {'id': 23, 'n

## Getting recommendations

The workforce planning service expects 

* an organisation ID (or list of user IDs in employee_ids)
* a list of dictionaries containing target profile IDs, number needed, and a maximum distance
* a graph_tag associated with the graph containing the skills used in the profiles



In [18]:
data = {"organisation_id":organisation_id,
        "target_profiles":[{"profile_id":1, # Applications Engineer
                            "number_needed":2,
                            "max_training":100},
                          {"profile_id":15, # Security Engineer
                            "number_needed":2,
                            "max_training":100},
                          {"profile_id":27, # Electrical Engineer
                            "number_needed":2,
                            "max_training":100}],
        "method":"distance",
        "graph_tag":graph_tag}

r = requests.post(base_url + "/workforce_planning", json=data)
print("Status code", r.status_code)

results = r.json()

Status code 200


In [23]:
# View results

print("Best match for each target")
for profile_id,matches in results['employees_by_target'].items():
    print(all_profiles[int(profile_id)])
    print("=" * len(all_profiles[int(profile_id)]))
    for user_id,m in matches:
        print("{0:<10}| {1}".format(all_users[user_id], m))
    print()

Best match for each target
Applications Engineer
Dani      | 30.0
Adam      | 15.0

Security Engineer
Dani      | 100.0
Adam      | 15.0

Electrical Engineer
Beth      | 100.0
Adam      | 0.0



In [24]:
# View results

print("Best match for each target")
for user_id,matches in results['targets_by_employee'].items():
    print(all_users[int(user_id)])
    print("=" * len(all_users[int(user_id)]))
    for profile_id,m in matches:
        print("{0:<10}| {1}".format(all_profiles[profile_id], m))
    print()

Best match for each target
Adam
====
Applications Engineer| 15.0
Security Engineer| 15.0
Electrical Engineer| 0.0

Beth
====
Electrical Engineer| 100.0
Applications Engineer| 0.0
Security Engineer| 0.0

Chris
=====
Applications Engineer| 0.0
Security Engineer| 0.0
Electrical Engineer| 0.0

Dani
====
Security Engineer| 100.0
Applications Engineer| 30.0
Electrical Engineer| 0.0

